# 使用scikit-learn实现逻辑回归

## 学习目标

通过本notebook，你将学会：
- 使用scikit-learn实现逻辑回归
- 处理二分类和多分类问题
- 使用正则化（L1和L2）
- 评估分类模型性能（准确率、ROC曲线、混淆矩阵）
- 理解多分类逻辑回归（Softmax）

## 课程概述

本notebook将展示如何使用scikit-learn库实现逻辑回归，包括：
1. **二分类问题**：基本的二分类任务
2. **多分类问题**：使用Softmax的多分类任务
3. **正则化**：L1和L2正则化的对比


## 1. 环境准备


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification, load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                            roc_curve, auc)
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

# 设置matplotlib在notebook中内联显示
%matplotlib inline

print("环境准备完成！")


## 2. 示例1：二分类问题

使用逻辑回归解决二分类问题，包括数据准备、模型训练、评估和可视化。


In [ ]:
# 生成二分类数据
X, y = make_classification(n_samples=1000, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1, 
                           random_state=42)

# 划分数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 标准化（逻辑回归通常需要标准化，特别是使用正则化时）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征维度: {X_train.shape[1]}")


In [ ]:
# 创建模型
# random_state: 随机种子，确保结果可复现
# max_iter: 最大迭代次数
model = LogisticRegression(random_state=42, max_iter=1000)

# 训练模型
model.fit(X_train_scaled, y_train)

# 预测
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # 正类的概率

# 评估
accuracy = accuracy_score(y_test, y_pred)
print(f"准确率: {accuracy:.4f}")
print("\n分类报告:")
print(classification_report(y_test, y_pred))

# 计算ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
print(f"\nROC AUC: {roc_auc:.4f}")

# 显示模型参数
print(f"\n模型参数:")
print(f"偏置项: {model.intercept_[0]:.4f}")
print(f"权重: {model.coef_[0]}")


In [ ]:
# 可视化结果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. ROC曲线
axes[0].plot(fpr, tpr, label=f'ROC曲线 (AUC = {roc_auc:.2f})', linewidth=2)
axes[0].plot([0, 1], [0, 1], 'k--', label='随机分类器')
axes[0].set_xlabel('假正例率 (FPR)')
axes[0].set_ylabel('真正例率 (TPR)')
axes[0].set_title('ROC曲线')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. 决策边界
x_min, x_max = X_test_scaled[:, 0].min() - 1, X_test_scaled[:, 0].max() + 1
y_min, y_max = X_test_scaled[:, 1].min() - 1, X_test_scaled[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))
Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z = Z.reshape(xx.shape)

axes[1].contourf(xx, yy, Z, levels=50, alpha=0.5, cmap='RdYlBu')
axes[1].contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
axes[1].scatter(X_test_scaled[:, 0], X_test_scaled[:, 1], c=y_test, 
               cmap='RdYlBu', edgecolors='black', s=30)
axes[1].set_xlabel('特征1')
axes[1].set_ylabel('特征2')
axes[1].set_title('决策边界')
plt.colorbar(axes[1].contourf(xx, yy, Z, levels=50, alpha=0.5, cmap='RdYlBu'), 
             ax=axes[1], label='预测概率')

plt.tight_layout()
plt.show()


## 3. 示例2：多分类问题

使用逻辑回归解决多分类问题（Iris数据集），展示Softmax回归。


In [ ]:
# 加载Iris数据集
iris = load_iris()
X, y = iris.data, iris.target

# 划分数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"数据集信息:")
print(f"  类别数量: {len(iris.target_names)}")
print(f"  类别名称: {iris.target_names}")
print(f"  特征数量: {X.shape[1]}")
print(f"  样本数量: {X.shape[0]}")


In [ ]:
# 创建多分类模型
# multi_class='multinomial': 使用Softmax（多项逻辑回归）
# solver='lbfgs': 优化算法，适合多分类问题
model_multi = LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                  random_state=42, max_iter=1000)

# 训练
model_multi.fit(X_train_scaled, y_train)

# 预测
y_pred = model_multi.predict(X_test_scaled)
y_proba = model_multi.predict_proba(X_test_scaled)  # 每个类别的概率

# 评估
accuracy = accuracy_score(y_test, y_pred)
print(f"准确率: {accuracy:.4f}")
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)
print(f"\n混淆矩阵:")
print(cm)


In [ ]:
# 可视化混淆矩阵
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.title('混淆矩阵')
plt.ylabel('真实标签')
plt.xlabel('预测标签')
plt.tight_layout()
plt.show()

# 显示前5个样本的概率预测
print("\n前5个样本的概率预测:")
print("样本 | 真实类别 | 预测类别 | 各类别概率")
print("-" * 50)
for i in range(min(5, len(y_test))):
    print(f"  {i+1}  |    {iris.target_names[y_test[i]]:<8} | "
          f"  {iris.target_names[y_pred[i]]:<8} | {y_proba[i]}")


## 4. 示例3：正则化对比

对比L1和L2正则化对模型性能的影响，使用乳腺癌数据集。


In [ ]:
# 加载乳腺癌数据集
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# 划分数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"数据集信息:")
print(f"  样本数量: {X.shape[0]}")
print(f"  特征数量: {X.shape[1]}")
print(f"  类别: {cancer.target_names}")


In [ ]:
# 测试不同的正则化方法
# C是正则化强度的倒数，C越小，正则化越强
regularizations = [
    ('L2 (C=1.0)', LogisticRegression(penalty='l2', C=1.0, random_state=42, max_iter=1000)),
    ('L2 (C=0.1)', LogisticRegression(penalty='l2', C=0.1, random_state=42, max_iter=1000)),
    ('L1 (C=1.0)', LogisticRegression(penalty='l1', C=1.0, solver='liblinear', 
                                      random_state=42, max_iter=1000)),
    ('L1 (C=0.1)', LogisticRegression(penalty='l1', C=0.1, solver='liblinear', 
                                      random_state=42, max_iter=1000)),
]

results = []
for name, model in regularizations:
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    # 统计非零参数数量（L1正则化会产生稀疏性）
    n_nonzero = np.sum(model.coef_[0] != 0)
    
    results.append({
        'name': name,
        'accuracy': accuracy,
        'n_nonzero': n_nonzero,
        'n_features': len(model.coef_[0])
    })
    
    print(f"{name:<15}: 准确率={accuracy:.4f}, "
          f"非零参数={n_nonzero}/{len(model.coef_[0])}")

# 可视化结果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 准确率对比
names = [r['name'] for r in results]
accuracies = [r['accuracy'] for r in results]
axes[0].bar(names, accuracies, color=['skyblue', 'lightblue', 'lightcoral', 'salmon'])
axes[0].set_ylabel('准确率')
axes[0].set_title('不同正则化方法的准确率')
axes[0].set_xticklabels(names, rotation=45, ha='right')
axes[0].grid(True, alpha=0.3)

# 非零参数数量（稀疏性）
n_nonzeros = [r['n_nonzero'] for r in results]
axes[1].bar(names, n_nonzeros, color=['skyblue', 'lightblue', 'lightcoral', 'salmon'])
axes[1].set_ylabel('非零参数数量')
axes[1].set_title('不同正则化方法的稀疏性（L1会产生稀疏性）')
axes[1].set_xticklabels(names, rotation=45, ha='right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n观察:")
print("- L1正则化会产生稀疏性（部分参数变为0），适合特征选择")
print("- L2正则化会缩小所有参数，但不为0")
print("- 正则化强度（C值）越小，正则化效果越强")


## 5. 总结与思考

### 关键知识点总结

1. **scikit-learn的LogisticRegression**：
   - 支持二分类和多分类
   - 默认使用L2正则化
   - 需要标准化特征

2. **多分类逻辑回归**：
   - 使用`multi_class='multinomial'`启用Softmax
   - 适合多类别分类问题

3. **正则化**：
   - L1正则化：产生稀疏性，适合特征选择
   - L2正则化：缩小所有参数，防止过拟合
   - C参数：正则化强度的倒数，C越小正则化越强

### 思考问题

1. **为什么逻辑回归需要标准化？**
   - 提示：考虑正则化和梯度下降的影响

2. **L1和L2正则化有什么区别？**
   - 提示：观察参数的变化和稀疏性

3. **什么时候使用多分类逻辑回归？**
   - 提示：考虑类别数量和问题类型

### 下一步学习

- 学习处理类别不平衡问题
- 学习特征工程和特征选择
- 学习模型调优和超参数搜索
